# NestedNameExtraction model(Chinese) to extract location, brand name, industry and forms of companies

Set Google Drive envrioment

In [ ]:
from google.colab import drive
from google.colab import files
drive.mount('/content/gdrive')

Start Git Clone PaddleNLP

In [ ]:
!git clone https://github.com/PaddlePaddle/PaddleNLP.git

Cloning into 'PaddleNLP'...
remote: Enumerating objects: 22221, done.
remote: Counting objects: 100% (302/302), done.
remote: Compressing objects: 100% (187/187), done.
remote: Total 22221 (delta 136), reused 224 (delta 102), pack-reused 21919
Receiving objects: 100% (22221/22221), 70.56 MiB | 10.74 MiB/s, done.
Resolving deltas: 100% (14357/14357), done.
Checking out files: 100% (2812/2812), done.


Install the GPU-based packages and Python environment

In [9]:
!python -m pip install paddlepaddle-gpu==2.3.0 -i https://mirror.baidu.com/pypi/simple
!python -c "import platform;print(platform.architecture()[0]);print(platform.machine())"
!pip install paddlenlp

Looking in indexes: https://mirror.baidu.com/pypi/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
64bit
x86_64


Check whether the package has been successfully installed

In [5]:
import paddle
paddle.utils.run_check()

Split the original dataset into training and validation datasets

In [7]:
!python /content/gdrive/MyDrive/project_folder/PaddleNLP/model_zoo/uie/doccano.py --doccano_file /content/gdrive/MyDrive/project_folder/PaddleNLP/paddlenlp/data/doccano_ext.json --task_type "ext" --save_dir /content/gdrive/MyDrive/project_folder/PaddleNLP/paddlenlp/data --splits 0.1 0.9 0

python3: can't open file '/content/gdrive/MyDrive/project_folder/PaddleNLP/model_zoo/uie/doccano.py': [Errno 2] No such file or directory


Tune the pre-trained model using training datasets(Location, Brand Name, Industry and Forms of Companies)

In [11]:
!python /content/gdrive/MyDrive/project_folder/PaddleNLP/model_zoo/uie/finetune.py --train_path /content/gdrive/MyDrive/project_folder/PaddleNLP/paddlenlp/data/train.txt --dev_path /content/gdrive/MyDrive/project_folder/PaddleNLP/paddlenlp/data/dev.txt --save_dir /content/gdrive/MyDrive/project_folder/PaddleNLP/paddlenlp/checkpoint --learning_rate 1e-5 --batch_size 16 --max_seq_len 512 --num_epochs 100 --model "uie-base" --seed 1000 --logging_steps 10 --valid_steps 100 --device "gpu"

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/flatbuffers/compat.py:19: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish

Deploy the fine-tuned model by setting schemas consistent with previous labels in the training dataset

In [12]:
from pprint import pprint
from paddlenlp import Taskflow
schema = ['地域', '商号', '行业', '组织形式']
# 设定抽取目标和定制化模型权重路径
my_ie = Taskflow("information_extraction", schema=schema, task_path='/content/gdrive/MyDrive/project_folder/PaddleNLP/paddlenlp/checkpoint/model_best/')

/usr/local/lib/python3.7/dist-packages/scipy/fft/__init__.py:97: DeprecationWarning: The module numpy.dual is deprecated.  Instead of using dual, use the functions directly from numpy or scipy.
  from numpy.dual import register_func
/usr/local/lib/python3.7/dist-packages/scipy/sparse/sputils.py:17: DeprecationWarning: `np.typeDict` is a deprecated alias for `np.sctypeDict`.
  supported_dtypes = [np.typeDict[x] for x in supported_dtypes]
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecati

Some Examples

In [ ]:
pprint(my_ie('福州鑫众利设备租赁有限公司'))

[{'商号': [{'end': 5,
          'probability': 0.9998734037377659,
          'start': 2,
          'text': '鑫众利'}],
  '地域': [{'end': 2,
          'probability': 0.9998571917130903,
          'start': 0,
          'text': '福州'}],
  '组织形式': [{'end': 13,
            'probability': 0.9994970711594249,
            'start': 9,
            'text': '有限公司'}],
  '行业': [{'end': 9,
          'probability': 0.9994508024781368,
          'start': 5,
          'text': '设备租赁'}]}]


In [ ]:
pprint(my_ie('北京金杜律师事务所'))

[{'商号': [{'end': 4,
          'probability': 0.9997910366100768,
          'start': 2,
          'text': '金杜'}],
  '地域': [{'end': 2,
          'probability': 0.9998480130536507,
          'start': 0,
          'text': '北京'}],
  '行业': [{'end': 9,
          'probability': 0.9698699083294358,
          'start': 4,
          'text': '律师事务所'}]}]


In [ ]:
pprint(my_ie('北京大成律师事务所'))

[{'商号': [{'end': 4,
          'probability': 0.9998167831881801,
          'start': 2,
          'text': '大成'}],
  '地域': [{'end': 2,
          'probability': 0.9998370462907928,
          'start': 0,
          'text': '北京'}],
  '组织形式': [{'end': 9,
            'probability': 0.6838094307634464,
            'start': 6,
            'text': '事务所'}],
  '行业': [{'end': 9,
          'probability': 0.5791944092317749,
          'start': 4,
          'text': '律师事务所'}]}]


In [ ]:
pprint(my_ie('无锡市自来水有限责任公司'))

[{'地域': [{'end': 3,
          'probability': 0.9998273922721523,
          'start': 0,
          'text': '无锡市'}],
  '组织形式': [{'end': 12,
            'probability': 0.9996631892337433,
            'start': 6,
            'text': '有限责任公司'}],
  '行业': [{'end': 6,
          'probability': 0.9992593065193702,
          'start': 3,
          'text': '自来水'}]}]


In [ ]:
pprint(my_ie('北京市晨曦科技有限公司'))

[{'商号': [{'end': 5,
          'probability': 0.9998004534898683,
          'start': 3,
          'text': '晨曦'}],
  '地域': [{'end': 3,
          'probability': 0.9997355488467363,
          'start': 0,
          'text': '北京市'}],
  '组织形式': [{'end': 11,
            'probability': 0.9997766675485451,
            'start': 7,
            'text': '有限公司'}],
  '行业': [{'end': 7,
          'probability': 0.9989371719875706,
          'start': 5,
          'text': '科技'}]}]


In [ ]:
pprint(my_ie('烟台南山游艇俱乐部有限公司'))

[{'商号': [{'end': 4,
          'probability': 0.9997869836411724,
          'start': 2,
          'text': '南山'}],
  '地域': [{'end': 2,
          'probability': 0.9998596949935745,
          'start': 0,
          'text': '烟台'}],
  '组织形式': [{'end': 13,
            'probability': 0.9996931051837166,
            'start': 9,
            'text': '有限公司'}],
  '行业': [{'end': 9,
          'probability': 0.998196618940483,
          'start': 4,
          'text': '游艇俱乐部'}]}]


In [13]:
pprint(my_ie("三千世界（昆山）文化产业投资合伙企业（有限合伙）"))

[{'商号': [{'end': 4,
          'probability': 0.9998122539286385,
          'start': 0,
          'text': '三千世界'}],
  '地域': [{'end': 7,
          'probability': 0.9997459810131772,
          'start': 5,
          'text': '昆山'}],
  '组织形式': [{'end': 24,
            'probability': 0.9724141175883183,
            'start': 19,
            'text': '有限合伙）'}],
  '行业': [{'end': 12,
          'probability': 0.8508985982037771,
          'start': 8,
          'text': '文化产业'}]}]


Multiple Inquries

In [ ]:
import pandas as pd
extraction = pd.read_excel("/content/gdrive/MyDrive/NameForExtraction.xlsx")

In [ ]:
extraction["Extraction"] = extraction["Name"].apply(lambda x: my_ie(x))

In [ ]:
from google.colab import files
extraction.to_csv('Extraction.csv', encoding = 'utf-8-sig') 
files.download('Extraction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>